# YOLO vs Optimized SAM Karşılaştırması
Bu notebook YOLO ile optimize edilmiş SAM sonuçlarını karşılaştırır.

## 1. Sistem Durumu ve Hazırlık

In [ ]:
import os
import subprocess
import time
import pandas as pd
import numpy as np
from datetime import datetime
import glob

# Çalışma dizinini ayarla
os.chdir('/workspace/line48/src')
print(f"📁 Çalışma dizini: {os.getcwd()}")

# Video dosyasını kontrol et
video_path = "../videos/short_video.mp4"
if os.path.exists(video_path):
    size = os.path.getsize(video_path) / (1024*1024)
    print(f"✅ Video hazır: {video_path} ({size:.1f} MB)")
else:
    print(f"❌ Video bulunamadı: {video_path}")

In [ ]:
# GPU durumunu kontrol et
!nvidia-smi --query-gpu=name,utilization.gpu,memory.used,memory.total,temperature.gpu --format=csv

## 2. Mevcut YOLO Sonuçlarını Kontrol Et

In [ ]:
# YOLO sonuçlarını bul ve göster
yolo_result_files = glob.glob('../logs/short_video_results_*.csv')
yolo_result_files = [f for f in yolo_result_files if 'sam' not in f]  # SAM dosyalarını hariç tut

if yolo_result_files:
    latest_yolo = max(yolo_result_files, key=os.path.getctime)
    print(f"📊 En son YOLO sonucu: {latest_yolo}")
    
    df_yolo = pd.read_csv(latest_yolo)
    print("\n🎯 YOLO Sonuçları:")
    print("=" * 40)
    
    yolo_total = 0
    for _, row in df_yolo.iterrows():
        cls = row.get('class', 'unknown')
        safe = row.get('safe_crossings', 0)
        uncertain = row.get('uncertain_crossings', 0)
        total = row.get('total_valid_crossings', 0)
        yolo_total += total
        print(f"  {cls:<10}: Safe={safe:>2}, Uncertain={uncertain:>2}, Total={total:>2}")
    
    print(f"  {'TOPLAM':<10}: {yolo_total:>2} geçiş")
    
    # YOLO processing time
    yolo_time = df_yolo['processing_time'].iloc[0] if 'processing_time' in df_yolo.columns else 'N/A'
    yolo_fps = df_yolo['avg_fps'].iloc[0] if 'avg_fps' in df_yolo.columns else 'N/A'
    print(f"\n⚡ YOLO Performance: {yolo_time}s, {yolo_fps} FPS")
    
else:
    print("❌ YOLO sonucu bulunamadı!")
    yolo_total = 0

## 3. Optimized SAM Analizini Çalıştır

In [ ]:
# Optimized SAM analizi başlat
print("🔬 Optimized SAM Analizi Başlatılıyor...")
print("="*60)
print("🎯 Optimizasyonlar:")
print("   - Min Safe Time: 3.0s (YOLO: 0.5s)")
print("   - Min Uncertain Time: 1.5s (YOLO: 0.28s)")
print("   - Confidence Threshold: 0.5 (YOLO: 0.25)")
print("   - Min Mask Area: 2000px")
print("   - Strict Tracking Distance: <50px")
print("="*60)

optimized_sam_start = time.time()

# Optimized SAM komutu
opt_sam_cmd = [
    "python", "run_sam_analysis_optimized.py",
    "--video", "../videos/short_video.mp4",
    "--sam-model", "vit_b",
    "--detailed-info"
]

print(f"🚀 Komut: {' '.join(opt_sam_cmd)}")
print("⏳ İşleniyor... (optimize edildi, daha makul sonuçlar bekleniyor)")

In [ ]:
# Optimized SAM'i çalıştır
try:
    opt_sam_result = subprocess.run(opt_sam_cmd, capture_output=True, text=True, timeout=1800)  # 30 dakika
    opt_sam_time = time.time() - optimized_sam_start
    
    print(f"⏱️  Optimized SAM İşleme Süresi: {opt_sam_time:.1f} saniye")
    
    if opt_sam_result.returncode == 0:
        print("✅ Optimized SAM analizi başarıyla tamamlandı!")
        print("\n📝 Optimized SAM Çıktısı:")
        print(opt_sam_result.stdout[-1500:])  # Son 1500 karakter
    else:
        print("❌ Optimized SAM analizi başarısız!")
        print("Hata:", opt_sam_result.stderr)
        
except subprocess.TimeoutExpired:
    print("⏰ Optimized SAM analizi 30 dakika sonra zaman aşımına uğradı")
except Exception as e:
    print(f"❌ Optimized SAM analizi hatası: {e}")

## 4. Optimized SAM Sonuçlarını Analiz Et

In [ ]:
# Optimized SAM sonuçlarını bul ve analiz et
opt_sam_result_files = glob.glob('../logs/*sam_optimized_results*.csv')

if opt_sam_result_files:
    latest_opt_sam = max(opt_sam_result_files, key=os.path.getctime)
    print(f"📊 Optimized SAM sonucu: {latest_opt_sam}")
    
    df_opt_sam = pd.read_csv(latest_opt_sam)
    print("\n🔬 Optimized SAM Sonuçları:")
    print("="*40)
    
    opt_sam_total = 0
    for _, row in df_opt_sam.iterrows():
        cls = row.get('class', 'unknown')
        safe = row.get('safe_crossings', 0)
        uncertain = row.get('uncertain_crossings', 0)
        total = row.get('total_valid_crossings', 0)
        opt_sam_total += total
        print(f"  {cls:<10}: Safe={safe:>2}, Uncertain={uncertain:>2}, Total={total:>2}")
    
    print(f"  {'TOPLAM':<10}: {opt_sam_total:>2} geçiş")
    
    # Optimized SAM performance
    opt_sam_proc_time = df_opt_sam['processing_time'].iloc[0] if 'processing_time' in df_opt_sam.columns else opt_sam_time
    opt_sam_fps = df_opt_sam['avg_fps'].iloc[0] if 'avg_fps' in df_opt_sam.columns else 'N/A'
    print(f"\n⚡ Optimized SAM Performance: {opt_sam_proc_time:.1f}s, {opt_sam_fps} FPS")
    
else:
    print("❌ Optimized SAM sonucu bulunamadı!")
    opt_sam_total = 0

## 5. Önceki SAM ile Karşılaştır

In [ ]:
# Önceki (optimize edilmemiş) SAM sonuçlarını bul
old_sam_files = glob.glob('../logs/*sam_results*.csv')
old_sam_files = [f for f in old_sam_files if 'optimized' not in f]  # Optimized olanları hariç tut

if old_sam_files:
    latest_old_sam = max(old_sam_files, key=os.path.getctime)
    print(f"📊 Önceki SAM sonucu: {latest_old_sam}")
    
    df_old_sam = pd.read_csv(latest_old_sam)
    print("\n🔬 Önceki SAM Sonuçları:")
    print("="*40)
    
    old_sam_total = 0
    for _, row in df_old_sam.iterrows():
        cls = row.get('class', 'unknown')
        safe = row.get('safe_crossings', 0)
        uncertain = row.get('uncertain_crossings', 0)
        total = row.get('total_valid_crossings', 0)
        old_sam_total += total
        print(f"  {cls:<10}: Safe={safe:>2}, Uncertain={uncertain:>2}, Total={total:>2}")
    
    print(f"  {'TOPLAM':<10}: {old_sam_total:>2} geçiş")
    
else:
    print("⚠️ Önceki SAM sonucu bulunamadı")
    old_sam_total = 930  # Bilinen değer

## 6. Kapsamlı Karşılaştırma Analizi

In [ ]:
# Kapsamlı karşılaştırma tablosu
print("📊 KAPSAMLI KARŞILAŞTIRMA ANALİZİ")
print("=" * 70)

# Özet tablo
print(f"{'Method':<20} {'Total Crossings':<15} {'Improvement':<12} {'Performance'}")
print("-" * 70)

# YOLO baseline
if yolo_total > 0:
    print(f"{'YOLO (Baseline)':<20} {yolo_total:<15} {'0%':<12} {yolo_time:.1f}s")
else:
    yolo_total = 18  # Bilinen değer
    print(f"{'YOLO (Baseline)':<20} {yolo_total:<15} {'0%':<12} {'39.6s'}")

# Önceki SAM
old_sam_improvement = ((old_sam_total - yolo_total) / yolo_total * 100) if yolo_total > 0 else 0
print(f"{'SAM (Original)':<20} {old_sam_total:<15} {old_sam_improvement:+.0f}%{'':<7} {'224.3s'}")

# Optimized SAM
if opt_sam_total > 0:
    opt_improvement = ((opt_sam_total - yolo_total) / yolo_total * 100) if yolo_total > 0 else 0
    print(f"{'SAM (Optimized)':<20} {opt_sam_total:<15} {opt_improvement:+.0f}%{'':<7} {opt_sam_proc_time:.1f}s")
else:
    print(f"{'SAM (Optimized)':<20} {'N/A':<15} {'N/A':<12} {'N/A'}")

print("-" * 70)

# Detaylı analiz
print(f"\n🔍 DETAYLI ANALİZ:")
print(f"\n📈 Doğruluk İyileştirmesi:")
print(f"   Original SAM:  {old_sam_improvement:+.0f}% (Çok abartılı!)")
if opt_sam_total > 0:
    print(f"   Optimized SAM: {opt_improvement:+.0f}% (Daha makul)")
    
    # Optimizasyon başarısı
    optimization_success = ((old_sam_total - opt_sam_total) / old_sam_total * 100) if old_sam_total > 0 else 0
    print(f"\n🎯 Optimizasyon Başarısı:")
    print(f"   SAM tespit sayısı {optimization_success:.0f}% azaldı")
    print(f"   {old_sam_total} → {opt_sam_total} geçiş")

print(f"\n⚡ Hız Karşılaştırması:")
print(f"   YOLO:          ~40s (En hızlı)")
print(f"   Original SAM:  ~224s (5.6x yavaş)")
if opt_sam_total > 0:
    print(f"   Optimized SAM: {opt_sam_proc_time:.0f}s ({opt_sam_proc_time/40:.1f}x yavaş)")

## 7. Sınıf Bazında Detaylı Karşılaştırma

In [ ]:
# Sınıf bazında detaylı karşılaştırma
print("\n📊 SINIF BAZINDA DETAYLI KARŞILAŞTIRMA")
print("=" * 60)

classes = ['person', 'backpack', 'handbag', 'suitcase']

print(f"{'Class':<12} {'YOLO':<8} {'Old SAM':<10} {'Opt SAM':<10} {'YOLO→Opt':<12}")
print("-" * 60)

for cls in classes:
    # YOLO değerleri
    if yolo_result_files:
        yolo_row = df_yolo[df_yolo['class'] == cls]
        yolo_val = yolo_row['total_valid_crossings'].iloc[0] if len(yolo_row) > 0 else 0
    else:
        # Bilinen değerler
        yolo_vals = {'person': 9, 'backpack': 6, 'handbag': 2, 'suitcase': 1}
        yolo_val = yolo_vals.get(cls, 0)
    
    # Old SAM değerleri
    if old_sam_files:
        old_sam_row = df_old_sam[df_old_sam['class'] == cls]
        old_sam_val = old_sam_row['total_valid_crossings'].iloc[0] if len(old_sam_row) > 0 else 0
    else:
        # Bilinen değerler
        old_sam_vals = {'person': 839, 'backpack': 5, 'handbag': 84, 'suitcase': 2}
        old_sam_val = old_sam_vals.get(cls, 0)
    
    # Optimized SAM değerleri
    if opt_sam_result_files:
        opt_sam_row = df_opt_sam[df_opt_sam['class'] == cls]
        opt_sam_val = opt_sam_row['total_valid_crossings'].iloc[0] if len(opt_sam_row) > 0 else 0
    else:
        opt_sam_val = 0
    
    # İyileşme hesapla
    if yolo_val > 0:
        improvement = ((opt_sam_val - yolo_val) / yolo_val * 100)
        improvement_str = f"{improvement:+.0f}%"
    else:
        improvement_str = "N/A"
    
    print(f"{cls:<12} {yolo_val:<8} {old_sam_val:<10} {opt_sam_val:<10} {improvement_str:<12}")

print("-" * 60)

# Toplam
total_yolo = yolo_total
total_old_sam = old_sam_total
total_opt_sam = opt_sam_total if opt_sam_total > 0 else 0

if total_yolo > 0:
    total_improvement = ((total_opt_sam - total_yolo) / total_yolo * 100)
    total_improvement_str = f"{total_improvement:+.0f}%"
else:
    total_improvement_str = "N/A"

print(f"{'TOTAL':<12} {total_yolo:<8} {total_old_sam:<10} {total_opt_sam:<10} {total_improvement_str:<12}")

## 8. Optimizasyon Başarı Raporu

In [ ]:
# Optimizasyon başarı raporu
print("\n🎯 OPTİMİZASYON BAŞARI RAPORU")
print("=" * 50)

if opt_sam_total > 0:
    # Başarı metrikleri
    reduction_rate = ((old_sam_total - opt_sam_total) / old_sam_total * 100) if old_sam_total > 0 else 0
    accuracy_to_yolo = abs(opt_sam_total - yolo_total) / yolo_total * 100 if yolo_total > 0 else 0
    
    print(f"✅ BAŞARILAR:")
    print(f"   📉 SAM tespitlerinde {reduction_rate:.0f}% azalma sağlandı")
    print(f"   📊 {old_sam_total} → {opt_sam_total} geçiş")
    print(f"   🎯 YOLO'ya yakınlık: {accuracy_to_yolo:.0f}% fark (Ideal: <50%)")
    
    # Başarı değerlendirmesi
    if accuracy_to_yolo < 50:
        print(f"   🏆 MÜKEMMEL: SAM artık YOLO'ya çok yakın sonuçlar veriyor!")
    elif accuracy_to_yolo < 100:
        print(f"   🟡 İYİ: SAM hala biraz yüksek ama çok daha makul")
    else:
        print(f"   🔴 DAHA İYİLEŞTİRME GEREKLİ: SAM hala çok fazla tespit yapıyor")
    
    print(f"\n📈 SONUÇ ÖNERİLERİ:")
    if accuracy_to_yolo < 30:
        print(f"   ✅ Optimized SAM kullanıma hazır!")
        print(f"   ✅ Piksel-level hassasiyet + makul sonuçlar")
    elif accuracy_to_yolo < 100:
        print(f"   🔧 Daha fazla fine-tuning gerekebilir")
        print(f"   🔧 Min safe time'ı daha da artırabilirsin")
    else:
        print(f"   ⚠️  Parametreler daha da sıkılaştırılmalı")
        print(f"   ⚠️  Confidence threshold'u 0.6'ya çıkar")
else:
    print(f"❌ Optimized SAM sonucu alınamadı")
    print(f"   Lütfen analizi tekrar çalıştır")

## 9. Video Çıktılarını Kontrol Et

In [ ]:
# Video dosyalarını listele ve boyutlarını kontrol et
print("🎬 OLUŞTURULAN VİDEO DOSYALARI")
print("=" * 50)

video_files = glob.glob('../outputs/short_video*.mp4')
video_files.extend(glob.glob('../outputs/short_video*.avi'))

for video in sorted(video_files):
    size = os.path.getsize(video) / (1024*1024)
    filename = os.path.basename(video)
    
    if 'yolo' in filename.lower() or ('sam' not in filename and 'processed' in filename):
        video_type = "🎯 YOLO"
    elif 'sam_optimized' in filename:
        video_type = "🔬 SAM (Optimized)"
    elif 'sam' in filename:
        video_type = "🔬 SAM (Original)"
    else:
        video_type = "❓ Unknown"
    
    print(f"  {video_type:<20} {filename} ({size:.1f} MB)")

print(f"\n📥 İndirme Talimatları:")
print(f"   # YOLO videosunu indir")
print(f"   scp root@your-vastai-ip:/workspace/line48/outputs/short_video_processed_*.mp4 ./yolo_result.mp4")
print(f"   ")
print(f"   # Optimized SAM videosunu indir")
print(f"   scp root@your-vastai-ip:/workspace/line48/outputs/short_video_sam_optimized_*.mp4 ./sam_optimized_result.mp4")

## 10. Final GPU Durumu ve Özet

In [ ]:
# Final GPU durumu
print("🎮 FINAL GPU DURUMU")
print("=" * 30)
!nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total,temperature.gpu --format=csv

# PyTorch memory temizle
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 GPU cache temizlendi")

# Final özet
print(f"\n🏁 FINAL ÖZET")
print(f"=" * 30)
print(f"✅ YOLO Analizi: {yolo_total} geçiş (~40s)")
print(f"❌ Original SAM: {old_sam_total} geçiş (~224s) - Çok abartılı")
if opt_sam_total > 0:
    print(f"🎯 Optimized SAM: {opt_sam_total} geçiş (~{opt_sam_proc_time:.0f}s) - Makul")
    
    if abs(opt_sam_total - yolo_total) / yolo_total < 0.5:
        print(f"\n🏆 SONUÇ: Optimized SAM başarıyla YOLO seviyesine yaklaştı!")
    else:
        print(f"\n🔧 SONUÇ: Optimized SAM iyileşti ama daha fazla tuning gerekli")
else:
    print(f"❓ Optimized SAM: Sonuç alınamadı")
    
print(f"\n🎉 Analiz tamamlandı!")